In [ ]:
import pyspark

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
import os
os.environ['JAVA_HOME'] = "/usr/java/jre1.8.0-x64"

In [ ]:
spark = SparkSession.builder.appName("joins_vsbroadcasting").getOrCreate()

In [ ]:
spark

In [ ]:
cust_df = spark.read.csv('customers.txt', sep='\t', header=True, inferSchema=True)

In [ ]:
cust_df.show(5)

In [ ]:
cust_df.printSchema()

In [ ]:
sales_df = spark.read.csv("salestxns.tsv", sep='\t', inferSchema=True)

In [ ]:
sales_df1 = sales_df.toDF("TxnId","CategoryId","CategoryName","ProductId","ProductName","Price","Quantity","customer_id")

In [ ]:
sales_df1.printSchema()

In [ ]:
cust_df.printSchema()

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType

In [ ]:
# Customised_schema = StructType([StructField("TxnID", IntegerType(), True),\
# StructField("Category_ID", IntegerType(), True]) \ .....

In [ ]:
sales_df1.show(4)

In [ ]:
spark.conf.get("spark.sql.autoBroadcastJoinThreshold")

In [ ]:
10485760/1024

In [ ]:
10240.0/1024

In [ ]:
# spark.conf.set("spark.sql.autoBroadcastJoinThreshold", 50*1024*1024)

In [ ]:
spark.conf.set("spark.sql.autoBroadcastJoinThreshold",-1)

In [ ]:
import datetime

In [ ]:
phys_exec_plan = sales_df1.join(cust_df, on = 'customer_id',how='left').explain()
start_time = datetime.datetime.now()
sales_df1.join(cust_df, on = 'customer_id',how='left').count()
end_time = datetime.datetime.now()
print("Processing time taken : ", end_time-start_time)

In [ ]:
spark.conf.set("spark.sql.autoBroadcastJoinThreshold",10485760)

In [ ]:
phys_exec_plan = sales_df1.join(broadcast(cust_df), on = 'customer_id',how='left').explain()
start_time = datetime.datetime.now()
sales_df1.join(broadcast(cust_df), on = 'customer_id',how='left').count()
end_time = datetime.datetime.now()
print("Processing time taken : ", end_time-start_time)

In [ ]:
from pyspark.sql.functions import broadcast

In [ ]:
without_broadcast = 1.17
with_broadcast = 0.11
process_time_reduced = without_broadcast-with_broadcast
pct = process_time_reduced/without_broadcast
print("The process time reduced with the help of broadcasting is :", pct)